In [55]:
import pandas as pd
import spotlight
from spotlight import exceptions as SpotlightException

def annotate_category(row):
    category_value = row['CATEGORY']

    try:
        # Annotate the category value using DBpedia Spotlight
        annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate?types={DBpedia:Food}', category_value, confidence=0.3)

        # Extract the URI of the 'category' property from the annotations
        category_uri = annotations[0]['URI'] if annotations else None

        print(row['index'])

        return category_uri

    except Exception as e:
        print(row['index'],' - not found')
        # Handle the exception (no resources found in spotlight response)
        return None

In [56]:
# Input and output file paths
csv_file_path = '../csvw/walmart_products_reduced.csv'
output_csv_file = '../csvw/walmart_products_reduced_annotated.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Apply the annotation function to each row and create the 'deducedType' column
df['DEDUCED_TYPE'] = df.apply(annotate_category, axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

0
17
34
51
68
85
102
119
136
153
170
187
204
221
238
255
272
289
306
323
340
357
374
391
408
425
442
459
476
493
510
527
544
561
578
595
612
629
646
663
680
697
714
731
748
765
782
799
816
833
850
867
884
901
918
935
952
969
986
1003
1020
1037
1054
1071
1088
1105
1122
1139
1156
1173
1190
1207
1224
1241
1258
1275
1292
1309
1326
1343
1360
1377
1394
1411
1428
1445
1462
1479
1496
1513
1530
1547
1564
1581
1598
1615
1632
1649
1666
1683
1700
1717
1734
1751
1768
1785
1802
1819
1836
1853
1870
1887
1904
1921
1938
1955
1972
1989
2006
2023
2040
2057
2074
2091
2108
2125
2142
2159
2176
2193
2210
2227
2244
2261
2278
2295
2312
2329
2346
2363
2380
2397
2414
2431
2448
2465
2482
2499
2516
2533
2550
2567
2584
2601
2618
2635
2652
2669
2686
2703
2720
2737
2754
2771
2788
2805
2822
2839
2856
2873
2890
2907
2924
2941
2958
2975
2992
3009
3026
3043
3060
3077
3094
3111
3128
3145
3162
3179
3196
3213
3230
3247
3264
3281
3298
3315
3332
3349
3366
3383
3400
3417
3434
3451
3468
3485
3502
3519
3536
3553
3570
3587
3604
3

In [2]:
%pip install pyspotlight

Note: you may need to restart the kernel to use updated packages.  Downloading pyspotlight-0.7.2-py2.py3-none-any.whl (15 kB)




[notice] A new release of pip is available: 23.1.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
